In [1]:
from datasets import SHD_dataloaders, SSC_dataloaders, GSC_dataloaders
from config import Config
from snn_delays import SnnDelays
import torch
from snn import SNN
import utils

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"\n=====> Device = {device} \n\n")


=====> Device = cpu 




In [3]:
config = Config()

if config.model_type == 'snn':
    model = SNN(config).to(device)
else:
    model = SnnDelays(config).to(device)

if config.model_type == 'snn_delays_lr0':
    model.round_pos()


print(f"===> Dataset    = {config.dataset}")
print(f"===> Model type = {config.model_type}")
print(f"===> Model size = {utils.count_parameters(model)}\n\n")

je suis là
===> Dataset    = shd
===> Model type = snn_dale
===> Model size = 214016




In [4]:
if config.dataset == 'shd':
    train_loader, valid_loader = SHD_dataloaders(config)
    test_loader = None
elif config.dataset == 'ssc':
    train_loader, valid_loader, test_loader = SSC_dataloaders(config)
elif config.dataset == 'gsc':
    train_loader, valid_loader, test_loader = GSC_dataloaders(config)
else:
    raise Exception(f'dataset {config.dataset} not implemented')

The directory [Datasets/SHD/extract] for saving extracted files already exists.
SpikingJelly will not check the data integrity of extracted files.
If extracted files are not integrated, please delete [Datasets/SHD/extract] manually, then SpikingJelly will re-extract files from [Datasets/SHD/download].
The directory [Datasets/SHD/duration_10] already exists.
The directory [Datasets/SHD/extract] for saving extracted files already exists.
SpikingJelly will not check the data integrity of extracted files.
If extracted files are not integrated, please delete [Datasets/SHD/extract] manually, then SpikingJelly will re-extract files from [Datasets/SHD/download].
The directory [Datasets/SHD/duration_10] already exists.


In [ ]:
#model.train_model(train_loader, valid_loader, test_loader, device)

In [5]:
# Get one batch from the train_loader
train_iter = iter(train_loader)
first_batch = next(train_iter)

In [6]:
# Assuming the batch is a tuple of (data, labels)
data, labels, _ = first_batch

# Print the first data point and its label
print(data[0], labels[0])

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 1.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]], dtype=torch.float64) tensor(0)


In [7]:
data.size()

torch.Size([256, 137, 140])

In [12]:
x, y = data[:, :30, :], data[:, 30:, :]

In [17]:
torch.cat((x, y), 0).size()

RuntimeError: Sizes of tensors must match except in dimension 0. Expected size 30 but got size 107 for tensor number 1 in the list.

In [ ]:
data = data.permute(1,0,2).float().to(device)  #(time, batch, neurons)
output = model.forward(data)

In [ ]:
output

In [ ]:
x = torch.rand(10, 2) > 0.2
x

In [ ]:
x[:, 1]

In [ ]:
model.train_model(train_loader, valid_loader, test_loader, device)

# **Nouvelle classe linéaire**

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

In [3]:
x = torch.tensor([1, 2, 3, 4, 5, 6, 7, 8, 9, 10])

In [8]:
x[:3]

tensor([1, 2, 3])

In [16]:
class DaleLinear(nn.Module):

    def __init__(self, in_features, out_features, exc_proportion):
        super().__init__()

        self.in_features_exc = round(in_features * exc_proportion)
        self.in_features_inh = in_features - self.in_features_exc

        self.w_exc = nn.Parameter(torch.rand(out_features, self.in_features_exc))
        self.w_inh = nn.Parameter(torch.rand(out_features, self.in_features_inh))
        self.bias = nn.Parameter(torch.rand(out_features, in_features))

    def forward(self, x):

        x_exc, x_inh = x[:, :self.in_features_exc, :]

        out_exc = F.linear(x_exc, torch.abs(self.w_exc), bias=False)
        out_inh = F.linear(x_inh, torch.abs(self.w_inh), bias=False)
        
        out = out_exc + out_inh + self.bias

        return out
    